In [1]:
!pip3 install numpy
!pip3 install pandas
!pip3 install seaborn
!pip3 install matplotlib
!pip3 install Jinja2


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: /usr/local/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: /usr/local/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: /usr/local/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: /usr/local/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: /usr/local/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #plotting
import seaborn as sns #good visualizing
import os
import warnings
warnings.filterwarnings('ignore')


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: /usr/local/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip


In [4]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

RuntimeError: Java gateway process exited before sending its port number

In [ ]:
import findspark
findspark.init()

In [ ]:
path = "TrainingDataset.csv"
validation_path = "ValidationDataset.csv"
data = spark.read.option("delimiter", ";").option("header", "true").option("inferSchema","true").csv(path)
validation_data = spark.read.option("delimiter", ";").option("header", "true").option("inferSchema","true").csv(validation_path)

In [ ]:
data.printSchema()

In [ ]:
data.summary().show()

In [ ]:
from pyspark.sql.functions import isnan, when, count, col

data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

In [ ]:
feature_list = ['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [ ]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

# convert to vector column first
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=feature_list, outputCol=vector_col)
df_vector = assembler.transform(data).select(vector_col)

# get correlation matrix
matrix = Correlation.corr(df_vector, vector_col).collect()[0][0] 
corr_matrix = matrix.toArray().tolist() 
corr_matrix_df = pd.DataFrame(data=corr_matrix, columns = feature_list, index=feature_list) 
corr_matrix_df .style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
data.columns

In [ ]:
train_assembler = VectorAssembler(inputCols=feature_list,outputCol="features")
df_train = assembler.transform(data)
df_train.show()

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = "corr_features", labelCol = 'quality')
rfModel = rf.fit(df_train)

In [ ]:
validation_assembler = VectorAssembler(inputCols=feature_list,outputCol="features")
df_validation = assembler.transform(validation_data)
df_validation.show()

In [ ]:
predictions = rfModel.transform(df_validation)
predictions.select('quality', 'rawPrediction', 'prediction', 'probability').show()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="quality", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F

preds_and_labels = predictions.select(['prediction','quality']).withColumn('quality', F.col('quality').cast(FloatType())).orderBy('prediction')
preds_and_labels = preds_and_labels.select(['prediction','quality'])
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
print(metrics.confusionMatrix().toArray())

# SVM